---

Grupo 9
* Bosch, Candelaria
* Juarez, Manuel
* Mecchia, Martín
* Montironi, Sebastian


---

# Laboratorio 2: Armado de un esquema de aprendizaje automático

En el laboratorio final se espera que puedan poner en práctica los conocimientos adquiridos en el curso, trabajando con un conjunto de datos de clasificación.

El objetivo es que se introduzcan en el desarrollo de un esquema para hacer tareas de aprendizaje automático: selección de un modelo, ajuste de hiperparámetros y evaluación.

El conjunto de datos a utilizar está en `./data/loan_data.csv`. Si abren el archivo verán que al principio (las líneas que empiezan con `#`) describen el conjunto de datos y sus atributos (incluyendo el atributo de etiqueta o clase).

Se espera que hagan uso de las herramientas vistas en el curso. Se espera que hagan uso especialmente de las herramientas brindadas por `scikit-learn`.

In [ ]:
import numpy as np
import pandas as pd

# TODO: Agregar las librerías que hagan falta
from sklearn.model_selection import train_test_split

## Carga de datos y división en entrenamiento y evaluación

La celda siguiente se encarga de la carga de datos (haciendo uso de pandas). Estos serán los que se trabajarán en el resto del laboratorio.

In [ ]:
#cargamos el csv para ejecutar en colab
#from google.colab import files
#uploaded = files.upload()

In [ ]:
dataset = pd.read_csv("loan_data.csv", comment="#")

# División entre instancias y etiquetas
X, y = dataset.iloc[:, 1:], dataset.TARGET

# división entre entrenamiento y evaluación
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


Documentación:

- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

## Ejercicio 1: Descripción de los Datos y la Tarea

Responder las siguientes preguntas:

1. ¿De qué se trata el conjunto de datos?
2. ¿Cuál es la variable objetivo que hay que predecir? ¿Qué significado tiene?
3. ¿Qué información (atributos) hay disponible para hacer la predicción?
4. ¿Qué atributos imagina ud. que son los más determinantes para la predicción?

**No hace falta escribir código para responder estas preguntas.**



1.   El conjunto de datos trata sobre el valor acumulado de la vivienda (HMEQ, por sus siglas en inglés) contiene referencia y rendimiento del préstamo e información de 5,960 préstamos recientes con garantía hipotecaria
2.   La variable objetivo (TARGET) que se busca predecir es si el cliente paga el credito o no. (Label: 1 = cliente  no paga el credito - 0 = credito pagado)
3.  Los atributos son:

  * LOAN: Importe de la solicitud de préstamo
  * MORTDUE: Monto adeudado sobre la hipoteca existente
  * VALUE: Valor de la propiedad actual
  * YOJ: Años en el trabajo actual
  * DEROG: Número de informes despectivos importantes
  * DELINQ: Número de líneas de crédito morosas
  * CLAGE: Antigüedad de la línea comercial más antigua en meses
  * NINQ: Número de líneas de crédito recientes
  * CLNO: Número de líneas de crédito
  * DEBTINC: Relación deuda-ingresos

4.  LOAN, MORTDUE, VALUE, DEROG, DELINQ, DEBTINC






## Ejercicio 2: Predicción con Modelos Lineales

En este ejercicio se entrenarán modelos lineales de clasificación para predecir la variable objetivo.

Para ello, deberán utilizar la clase SGDClassifier de scikit-learn.

Documentación:
- https://scikit-learn.org/stable/modules/sgd.html
- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html


### Ejercicio 2.1: SGDClassifier con hiperparámetros por defecto

Entrenar y evaluar el clasificador SGDClassifier usando los valores por omisión de scikit-learn para todos los parámetros. Únicamente **fijar la semilla aleatoria** para hacer repetible el experimento.

Evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

clf = SGDClassifier(random_state=0)

# Entrenamos el clasificador
clf.fit(X_train, y_train)

# Realizamos predicciones en el conjunto de entrenamiento y test
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

# Calculamos las métricas
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

train_precision = precision_score(y_train, y_train_pred)
test_precision = precision_score(y_test, y_test_pred)

train_recall = recall_score(y_train, y_train_pred)
test_recall = recall_score(y_test, y_test_pred)

train_f1 = f1_score(y_train, y_train_pred)
test_f1 = f1_score(y_test, y_test_pred)

train_confusion_matrix = confusion_matrix(y_train, y_train_pred)
test_confusion_matrix = confusion_matrix(y_test, y_test_pred)

# Imprimimos las métricas
print("Métricas para el conjunto de entrenamiento:")
print("Accuracy:", train_accuracy)
print("Precision:", train_precision)
print("Recall:", train_recall)
print("F1-score:", train_f1)
print("Matriz de confusión:")
print(train_confusion_matrix)
print()

print("Métricas para el conjunto de testeo:")
print("Accuracy:", test_accuracy)
print("Precision:", test_precision)
print("Recall:", test_recall)
print("F1-score:", test_f1)
print("Matriz de confusión:")
print(test_confusion_matrix)

### Ejercicio 2.2: Ajuste de Hiperparámetros

Seleccionar valores para los hiperparámetros principales del SGDClassifier. Como mínimo, probar diferentes funciones de loss, tasas de entrenamiento y tasas de regularización.

Para ello, usar grid-search y 5-fold cross-validation sobre el conjunto de entrenamiento para explorar muchas combinaciones posibles de valores.

Reportar accuracy promedio y varianza para todas las configuraciones.

Para la mejor configuración encontrada, evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión

Documentación:
- https://scikit-learn.org/stable/modules/grid_search.html
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [ ]:
from sklearn.model_selection import GridSearchCV

# Definimos los posibles valores para los hiperparámetros
param_grid = {
    'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron', 'huber', 'epsilon_insensitive'],
    'learning_rate': ['constant', 'optimal', 'invscaling'],
    'alpha': [0.0001, 0.001, 0.01, 0.1]
}

# Creamos el clasificador SGDClassifier
clf = SGDClassifier(random_state=0)

# Realizamos la búsqueda con validación cruzada
grid_search = GridSearchCV(clf, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Obtenemos los resultados de la búsqueda
results = grid_search.cv_results_
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Resultados de la búsqueda en cuadrícula:")
for mean_score, params in zip(results["mean_test_score"], results["params"]):
    print("Accuracy promedio:", mean_score)
    print("Configuración:", params)
    print()

print("Mejor configuración encontrada:")
print("Accuracy promedio:", best_score)
print("Hiperparámetros:", best_params)
print()

# Evaluamos el modelo con la mejor configuración en los conjuntos de entrenamiento y test
best_model = grid_search.best_estimator_
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

train_precision = precision_score(y_train, y_train_pred)
test_precision = precision_score(y_test, y_test_pred)

train_recall = recall_score(y_train, y_train_pred)
test_recall = recall_score(y_test, y_test_pred)

train_f1 = f1_score(y_train, y_train_pred)
test_f1 = f1_score(y_test, y_test_pred)

train_confusion_matrix = confusion_matrix(y_train, y_train_pred)
test_confusion_matrix = confusion_matrix(y_test, y_test_pred)

# Imprimimos las métricas para la mejor configuración encontrada
print("Métricas de evaluación para la mejor configuración encontrada:")
print("Accuracy en entrenamiento:", train_accuracy)
print("Accuracy en evaluación:", test_accuracy)
print("Precision en entrenamiento:", train_precision)
print("Precision en evaluación:", test_precision)
print("Recall en entrenamiento:", train_recall)
print("Recall en evaluación:", test_recall)
print("F1-score en entrenamiento:", train_f1)
print("F1-score en evaluación:", test_f1)
print("Matriz de confusión en entrenamiento:")
print(train_confusion_matrix)
print("Matriz de confusión en evaluación:")
print(test_confusion_matrix)

## Ejercicio 3: Árboles de Decisión

En este ejercicio se entrenarán árboles de decisión para predecir la variable objetivo.

Para ello, deberán utilizar la clase DecisionTreeClassifier de scikit-learn.

Documentación:
- https://scikit-learn.org/stable/modules/tree.html
  - https://scikit-learn.org/stable/modules/tree.html#tips-on-practical-use
- https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
- https://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html

### Ejercicio 3.1: DecisionTreeClassifier con hiperparámetros por defecto

Entrenar y evaluar el clasificador DecisionTreeClassifier usando los valores por omisión de scikit-learn para todos los parámetros. Únicamente **fijar la semilla aleatoria** para hacer repetible el experimento.

Evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión


In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(random_state=0)

# Entrenamos el modelo
clf.fit(X_train, y_train)

# Realizamos predicciones en los conjuntos de entrenamiento y testeo
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

# Calculamos las métricas
accuracy_train = accuracy_score(y_train, y_train_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)

precision_train = precision_score(y_train, y_train_pred)
precision_test = precision_score(y_test, y_test_pred)

recall_train = recall_score(y_train, y_train_pred)
recall_test = recall_score(y_test, y_test_pred)

f1_train = f1_score(y_train, y_train_pred)
f1_test = f1_score(y_test, y_test_pred)

confusion_matrix_train = confusion_matrix(y_train, y_train_pred)
confusion_matrix_test = confusion_matrix(y_test, y_test_pred)

# Imprimimos los resultados
print("Resultados en el conjunto de entrenamiento:")
print(f"Accuracy: {accuracy_train:.4f}")
print(f"Precision: {precision_train:.4f}")
print(f"Recall: {recall_train:.4f}")
print(f"F1: {f1_train:.4f}")
print("Matriz de confusión:")
print(confusion_matrix_train)
print()

print("Resultados en el conjunto de test:")
print(f"Accuracy: {accuracy_test:.4f}")
print(f"Precision: {precision_test:.4f}")
print(f"Recall: {recall_test:.4f}")
print(f"F1: {f1_test:.4f}")
print("Matriz de confusión:")
print(confusion_matrix_test)

### Ejercicio 3.2: Ajuste de Hiperparámetros

Seleccionar valores para los hiperparámetros principales del DecisionTreeClassifier. Como mínimo, probar diferentes criterios de partición (criterion), profundidad máxima del árbol (max_depth), y cantidad mínima de samples por hoja (min_samples_leaf).

Para ello, usar grid-search y 5-fold cross-validation sobre el conjunto de entrenamiento para explorar muchas combinaciones posibles de valores.

Reportar accuracy promedio y varianza para todas las configuraciones.

Para la mejor configuración encontrada, evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión


Documentación:
- https://scikit-learn.org/stable/modules/grid_search.html
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [ ]:
# Definimos los hiperparámetros a explorar en el grid search
parameters = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15],
    'min_samples_leaf': [1, 2, 3, 4]
}

# Creamos el clasificador DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)

# Realizamos el grid search con 5-fold cross-validation
grid_search = GridSearchCV(clf, parameters, cv=5)
grid_search.fit(X_train, y_train)

# Obtenemos los resultados del grid search
results = pd.DataFrame(grid_search.cv_results_)
best_params = grid_search.best_params_

# Imprimimos los resultados del grid search
print("Resultados del Grid Search:")
print(results[['params', 'mean_test_score', 'std_test_score']])

# Obtenemos el mejor modelo encontrado
best_model = grid_search.best_estimator_

# Evaluamos el mejor modelo en los conjuntos de entrenamiento y test
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

accuracy_train = accuracy_score(y_train, y_train_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)

precision_train = precision_score(y_train, y_train_pred)
precision_test = precision_score(y_test, y_test_pred)

recall_train = recall_score(y_train, y_train_pred)
recall_test = recall_score(y_test, y_test_pred)

f1_train = f1_score(y_train, y_train_pred)
f1_test = f1_score(y_test, y_test_pred)

confusion_matrix_train = confusion_matrix(y_train, y_train_pred)
confusion_matrix_test = confusion_matrix(y_test, y_test_pred)

# Imprimimos los resultados del mejor modelo
print("\nMejor configuración encontrada:")
print(best_params)
print()

print("Resultados en el conjunto de entrenamiento:")
print(f"Accuracy: {accuracy_train:.4f}")
print(f"Precision: {precision_train:.4f}")
print(f"Recall: {recall_train:.4f}")
print(f"F1: {f1_train:.4f}")
print("Matriz de confusión:")
print(confusion_matrix_train)
print()

print("Resultados en el conjunto de testeo:")
print(f"Accuracy: {accuracy_test:.4f}")
print(f"Precision: {precision_test:.4f}")
print(f"Recall: {recall_test:.4f}")
print(f"F1: {f1_test:.4f}")
print("Matriz de confusión:")
print(confusion_matrix_test)